In [76]:
import os
import io
import yaml
import json
from datetime import datetime, timedelta

import pandas as pd
import psycopg2
from sshtunnel import SSHTunnelForwarder
import boto3

In [22]:
with open('private/secrets.yml') as in_file:
    secrets = yaml.load(in_file)

In [23]:
s3 = boto3.resource('s3',
    aws_access_key_id=secrets['aws_access_key'],
    aws_secret_access_key=secrets['aws_secret_access_key'],
)

In [3]:
def query(q):
    with SSHTunnelForwarder(
        ('hkgmetadb.infra.ooni.io', 22),
        ssh_username='art',
        ssh_private_key=secrets['ssh_private_key_path'],
        remote_bind_address=('localhost', 5432)
    ) as server:
        conn = psycopg2.connect(
            host='localhost',
            port=server.local_bind_port,
            user='shovel',
            password=secrets['shovel_password'],
            dbname='metadb')
        return pd.read_sql_query(q, conn)

# Required metrics


### Homepage overview

* Total number of measurements collected (see: `msmt_count`)
* Total number of countries covered by measurements (see: `country_count`)
* Total number of networks covered by measurements

## Country page

* Overall number of measurements for this country
* Overall number of networks covered by measurements
* Number of networks on which we found a middlebox (MISSING-EXTRACTOR)
* If Instant Messagging apps may be blocked (MISSING-EXTRACTOR)
* If circumvention tools may be blocked (MISSING-EXTRACTOR)
* The number of blocked sites per category (we have the extractor, but we don't have the category codes)
* Daily measurement count per nettest group (see: `get_30day_measurement_count()`)
* Daily network count per nettest group (see: `get_30day_test_group_count_by_network()`)

### Performance

* Median upload speed (MISSING-EXTRACTOR)
* Median upload speed (MISSING-EXTRACTOR)

### Instant Messaging

For each app (`whatsapp`, `telegram`, `facebook_messenger`) **and network**:

* Number of measurements
* App status verdict (MISSING-EXTRACTOR)
* Timestamp of the last test

### Middleboxes

List of networks tested and for each:
* Number of measurements
* If we found a middlebox (MISSING-EXTRACTOR)
* Timestamp of the last test

### Websites

List of URLs that were tested and for each:

* Category code
* Number of measurements showing blocking
* Number of measurements showing an anomaly
* Overall number of measurements
* Timestamp of the last test

XXX we may want to have the window of this be adjustable.

In [5]:
%%time
msmt_count = query("SELECT COUNT(msm_no) FROM measurement;")

CPU times: user 514 ms, sys: 291 ms, total: 805 ms
Wall time: 8min 2s


In [17]:
%%time
country_count = query("SELECT COUNT(DISTINCT probe_cc) FROM report;")

CPU times: user 37.7 ms, sys: 9.69 ms, total: 47.4 ms
Wall time: 13.2 s


In [18]:
%%time
network_count = query("SELECT COUNT(DISTINCT probe_asn) FROM report;")

CPU times: user 31.4 ms, sys: 7.23 ms, total: 38.7 ms
Wall time: 5.69 s


In [29]:
obj = {
    'measurement_count': int(msmt_count['count'].iloc[0]),
    'country_count': int(country_count['count'].iloc[0]),
    'network_count': int(network_count['count'].iloc[0]),
}

In [36]:
with open('exports/homepage-overview.json', 'w') as out_file:
    json.dump(obj, out_file)

In [38]:
s3.Bucket('ooni-explorer').put_object(
    Key='static/homepage-overview.json',
    Body=open('exports/homepage-overview.json', 'rb')
)

s3.Object(bucket_name='ooni-explorer', key='static/homepage-overview.json')

In [39]:
%%time
country_list = query("SELECT DISTINCT probe_cc FROM report;")

CPU times: user 25.5 ms, sys: 8.57 ms, total: 34.1 ms
Wall time: 6.03 s


In [41]:
%%time
counts_by_country = query("""SELECT
	COUNT(msm_no) as msmt_count,
	probe_cc
FROM measurement 
JOIN report ON report.report_no = measurement.report_no
GROUP BY probe_cc;
""")

CPU times: user 383 ms, sys: 186 ms, total: 570 ms
Wall time: 7min 51s


In [43]:
%%time
network_count_by_country = query("""SELECT
	COUNT(DISTINCT probe_asn) as network_count,
	probe_cc
FROM report
GROUP BY probe_cc;""")

CPU times: user 52.2 ms, sys: 18.4 ms, total: 70.6 ms
Wall time: 40.2 s


CPU times: user 534 ms, sys: 269 ms, total: 804 ms
Wall time: 8min 49s


In [ ]:
# This window should be adjusted in function of how long it takes the pipeline to process measurements
start_date = (datetime.utcnow() - timedelta(days=30) - timedelta(hours=48)).strftime('%Y-%m-%d')
end_date = (datetime.utcnow() - timedelta(hours=48)).strftime('%Y-%m-%d')

In [106]:
print('interval: {} - {}'.format(start_date, end_date))

interval: 2018-05-06 - 2018-06-05


In [64]:
%%time
pipeline_30day_window = """
test_start_time >= {start_date}
AND test_start_time <= {end_date}
""".format(start_date=start_date, end_date=end_date)

test_count_30days = query("""
SELECT
    COUNT(msm_no) as msmt_count,
    probe_cc,
    probe_asn,
    test_name,
    date_trunc('day', test_start_time) as test_day
FROM measurement 
JOIN report ON report.report_no = measurement.report_no
WHERE {}
GROUP BY probe_cc, probe_asn, test_name, test_day;""".format(pipeline_30day_window))

CPU times: user 1.07 s, sys: 556 ms, total: 1.62 s
Wall time: 14min 22s


In [241]:
%%time
confirmed_blocked_sites = query("""SELECT DISTINCT input,
probe_cc
 FROM measurement 
JOIN input ON input.input_no = measurement.input_no 
JOIN report ON report.report_no = measurement.report_no
WHERE confirmed = true
GROUP BY probe_cc, input;
""")

CPU times: user 54.3 ms, sys: 52.5 ms, total: 107 ms
Wall time: 16.4 s


In [171]:
test_names = [
    # instant_messaging
    'facebook_messenger',
    'whatsapp',
    'telegram',

    # middleboxes
    'http_header_field_manipulation',
    'http_invalid_request_line',
    
    # performance
    'dash',
    'ndt',
    
    # websites
    'web_connectivity',
    
    # circumvention
    'vanilla_tor',

    # Other
    'http_requests',
    'meek_fronted_requests_test',
    'multi_protocol_traceroute',
    'tcp_connect'
]
test_groups = {
    # instant_messaging
    'facebook_messenger': 'instant_messaging',
    'whatsapp': 'instant_messaging',
    'telegram': 'instant_messaging',

    # middleboxes
    'http_header_field_manipulation': 'middleboxes',
    'http_invalid_request_line': 'middleboxes',
    
    # performance
    'dash': 'performance',
    'ndt': 'performance',
    
    # websites
    'web_connectivity': 'websites',
    
    # circumvention
    'vanilla_tor': 'circumvention',

    # other
    'http_requests': 'other',
    'meek_fronted_requests_test': 'other',
    'multi_protocol_traceroute': 'other',
    'tcp_connect': 'other'
}

In [ ]:
def get_blocked_websites():
    pass

In [238]:
def get_30day_measurement_count(probe_cc, start_date=start_date, end_date=end_date):
    df = test_count_30days[test_count_30days['probe_cc'] == probe_cc].copy()

    if df.size > 0:
        # Add the test_group column
        df['test_group'] = df.apply(lambda x: test_groups.get(x['test_name'], 'other'), axis=1)
    else:
        # Just add an empty column when we have now rows
        df['test_group'] = 0
    # We count the number of measurements every day for every test_group
    df = df.sort_values(by='test_day')[['test_group', 'test_day', 'msmt_count']]\
        .groupby(['test_group', 'test_day']).sum()
    # We ensure the time interval index has no "holes" by reindexing it based on the start and end_date
    df = df.unstack(level=0)\
        .reindex(pd.date_range(start=start_date, end=end_date))

    # Remove the multi-index from the columns
    df.columns = df.columns.droplevel()
    test_group_names = list(set(test_groups.values()))
    # Reindex the columns as well to ensure we don't have any missing column
    return df.reindex(columns=test_group_names).fillna(0)

In [239]:
def get_30day_test_group_count_by_network(probe_cc, start_date=start_date, end_date=end_date):
    df = test_count_30days[test_count_30days['probe_cc'] == probe_cc].copy()
    
    if df.size > 0:
        # Add the test_group column
        df['test_group'] = df.apply(lambda x: test_groups.get(x['test_name'], 'other'), axis=1)
    else:
        # Just add an empty column when we have now rows
        df['test_group'] = 0
    # We count the number of networks covered every day for every test_name
    df = df.sort_values(by='test_day')[['test_group', 'test_day', 'probe_asn']]\
            .groupby(['test_group', 'test_day']).count()

    # We ensure the time interval index has no "holes" by reindexing it based on the start and end_date
    df = df.unstack(level=0)\
        .reindex(pd.date_range(start=start_date, end=end_date))
    
    # Remove the multi-index from the columns
    df.columns = df.columns.droplevel()
    test_group_names = list(set(test_groups.values()))
    # Reindex the columns as well to ensure we don't have any missing column
    return df.reindex(columns=test_group_names).fillna(0)

In [240]:
for probe_cc in list(country_list['probe_cc']):
    dst_dir = os.path.join('exports', 'explorer', probe_cc)
    try:
        os.makedirs(dst_dir)
    except:
        # Ignore existing
        pass
    try:
        get_30day_test_group_count_by_network(probe_cc).to_json(
            os.path.join(dst_dir, 'test_group_count_by_network_30days-{}.json'.format(probe_cc)),
            date_format='iso'
        )
        get_30day_measurement_count(probe_cc).to_json(
            os.path.join(dst_dir, 'measurement_count_30days-{}.json'.format(probe_cc)),
            date_format='iso'
        )
    except Exception as exc:
        print(probe_cc)
        raise exc

In [60]:
networks_per_test_count_30days[
    networks_per_test_count_30days['probe_cc'] == 'US'
].sort_values(by='test_day')[['probe_asn', 'test_day']].groupby('test_day').count().head()

,probe_asn
test_day,
2018-05-07,76
2018-05-08,70
2018-05-09,68
2018-05-10,78
2018-05-11,89


In [63]:
msmt_per_test_count_30days[
    msmt_per_test_count_30days['probe_cc'] == 'US'
].sort_values(by='test_day')

,msmt_count,probe_cc,test_name,test_day
5244,27,US,whatsapp,2018-05-07
5842,4,US,meek_fronted_requests_test,2018-05-07
17712,112,US,http_header_field_manipulation,2018-05-07
4177,102,US,http_invalid_request_line,2018-05-07
1591,306,US,tcp_connect,2018-05-07
7048,35,US,facebook_messenger,2018-05-07
8277,315,US,ndt,2018-05-07
11757,81,US,dash,2018-05-07
11283,2,US,vanilla_tor,2018-05-07
9231,26,US,telegram,2018-05-07
